# Development Transportation Mitigation
This notebook imports, cleans, and exports sample data on a series of developments from the SF Planning Department. These developments have a series of recommended transportation demand mitigation measures. This notebook imports, cleans, and exports this data for a web map in Javascript. 

# Import Data and Packages

In [ ]:
import pandas as pd
import time
from geopy.geocoders import GoogleV3 #google geocoding
import requests  # library for accessing content from web URLs
import json    # library for working with JSON-formatted text strings
pd.set_option('display.max_row', 1000) #sets max rows to be seen
pd.set_option('display.max_columns', 50) #Set max column width to 50

In [ ]:
import_file = "Mapping.xlsx"
df = pd.read_excel(import_file)

In [ ]:
#keep rows that are actual addresses (i.e. nonmissing)
df = df[df['PROJECT ADDRESS'].notnull()]

In [ ]:
#Create full address field for geocoding
df['address'] = df['PROJECT ADDRESS'] + ", San Francisco, CA"

# Geocoding

In [ ]:
# set the pause duration between api requests
pause = 0.1

In [ ]:
# function that accepts an address string, sends it to the Google API, and returns the lat-long API result
def geocode(address):
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = 'http://maps.googleapis.com/maps/api/geocode/json?address={}&sensor=false' #api url with placeholders
    request = url.format(address) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = response.json() #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract lat-long from result
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        status = data['status'] #return geocode status
        return '{},{},{}'.format(latitude, longitude, status) #return lat-long as a string in the format google likes

In [ ]:
# for each value in the address column, geocode it, save results as new df column
start = time.time()
df['latlng'] = df['address'].map(geocode)
df['lat'] = df['latlng'].str.split(',').str[0]
df['lon'] = df['latlng'].str.split(',').str[1]
df['geocode_status'] = df['latlng'].str.split(',').str[2]
end = time.time()
print(end - start)

In [ ]:
#manually code two lat-lons that were not proper addresses
df.loc[df['PROJECT ADDRESS'] == 'Pier 70 -Waterfront Site (Forest City)', 'lat'] = 37.759562
df.loc[df['PROJECT ADDRESS'] == 'Pier 70 -Waterfront Site (Forest City)', 'lon'] = -122.382927
df.loc[df['PROJECT ADDRESS'] == 'Crane Cove Park/19th Street Extension', 'lat'] = 37.762979
df.loc[df['PROJECT ADDRESS'] == 'Crane Cove Park/19th Street Extension', 'lon'] = -122.386710

# Cleaning before export

In [ ]:
#Manually clean up some columns
df.loc[df['Consultant'] == 'Fehr&Peers', 'Consultant'] = 'Fehr & Peers'
df.loc[df['Significant Impacts if any?'] == 'No ', 'Significant Impacts if any?'] = 'No'

In [ ]:
# Rename categories for easier manipulation in Javascript
key = {'CASE #': 'case_number', 'PROJECT ADDRESS': 'Address', 'TIS/Circ Memo?': 'TIS_Circ_Memo', 'Significant Impacts if any?': 'sig_impacts', 
      'Recommended Mitigation Measures': 'mitigation_measures', 'Recommended Improvement Measures': 'improvement_measures', 
      'Recommended TDM Measures': 'TDM_measures'}
df.rename(columns=key, inplace=True)

In [ ]:
#Clean NaNs in some variables for map
list = ['mitigation_measures', 'improvement_measures', 'TDM_measures']
for i in list: 
    df[i].fillna("None", inplace = True)

# Export to GeoJSON

In [ ]:
#write function for dataframe
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [ ]:
# get rid of unwanted columns and then write in GeoJSON format
cols = ['case_number', 'Address', 'TIS_Circ_Memo', 'sig_impacts', 'Consultant', 'mitigation_measures', 'improvement_measures', 'TDM_measures', 'lat', 
       'lon']

geojson = df_to_geojson(df, cols, 'lat', 'lon')

In [ ]:
# save the geojson result to a file
output_filename = 'data.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson, indent=4)))